In [ ]:
# Import our dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the diabetes_w_in_24hr.csv.
import pandas as pd
diabetes_df = pd.read_csv('https://raw.githubusercontent.com/ComfyKoala/diabetes-classification/main/Nikko/Resources/Diabetes_w_in_24hr.csv')
# Display the first few rows
diabetes_df.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,...,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


# Clean, Standardize, and Encode Categorical Data

In [ ]:
# remove extra info
# reduced dataset to measurements taken within 24 hours (as opposed to only within the first hour after admission)
# simplified invasive/noninvasive/combined measurements to only the combined measurements
clean_diabetes_df = diabetes_df.drop(columns=['Unnamed: 0', 'encounter_id', 'hospital_id', 'elective_surgery', 'icu_admit_source', 'icu_id',
                                              'icu_stay_type', 'icu_type', 'pre_icu_los_days', 'readmission_status', 'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
                                              'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min',
                                              'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_sysbp_invasive_max',
                                              'd1_sysbp_invasive_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_invasive_max',
                                              'h1_diasbp_invasive_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
                                              'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_invasive_max', 'h1_mbp_invasive_min', 'h1_mbp_max', 'h1_mbp_min',
                                              'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min',
                                              'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max',
                                              'h1_sysbp_noninvasive_min', 'h1_temp_max', 'h1_temp_min', 'h1_albumin_max', 'h1_albumin_min', 'h1_bilirubin_max',
                                              'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min', 'h1_creatinine_max',
                                              'h1_creatinine_min', 'h1_glucose_max', 'h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max',
                                              'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max',
                                              'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min', 'h1_potassium_max', 'h1_potassium_min', 'h1_sodium_max',
                                              'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min', 'h1_arterial_pco2_max', 'h1_arterial_pco2_min', 'h1_arterial_ph_max',
                                              'h1_arterial_ph_min', 'h1_arterial_po2_max', 'h1_arterial_po2_min', 'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'solid_tumor_with_metastasis'])



In [ ]:
# count nulls, remove columns where null values exist for > 5000 of the rows
null_col_check = clean_diabetes_df.isnull().sum(axis=0)
null_col_check = null_col_check[null_col_check != 0][null_col_check > 5000]
remove_cols = []
for i, v in null_col_check.items():
  remove_cols.append(i)

clean_diabetes_df = clean_diabetes_df.drop(columns=remove_cols)

In [ ]:
# then decide which columns to remove altogether and which rows to remove due to nulls
null_row_check = clean_diabetes_df.isnull().sum(axis=1)
null_row_check = null_row_check[null_row_check != 0]
remove_rows = []
for i, v in null_row_check.items():
  remove_rows.append(i)

clean_diabetes_df = clean_diabetes_df.drop(remove_rows, axis=0)

In [ ]:
# scale numerical columns due to large differences in values
scaled_data = StandardScaler().fit_transform(clean_diabetes_df[['age', 'bmi', 'height', 'weight', 'gcs_eyes_apache', 'gcs_motor_apache',
                                                                'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'map_apache',
                                                                'resprate_apache', 'temp_apache', 'd1_diasbp_max', 'd1_diasbp_min',
                                                                'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min',
                                                                'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min',
                                                                'd1_temp_max', 'd1_temp_min', 'd1_glucose_max', 'd1_glucose_min']])
scaled_data_df = pd.DataFrame(scaled_data, columns=['age', 'bmi', 'height', 'weight', 'gcs_eyes_apache', 'gcs_motor_apache',
                                                    'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'map_apache',
                                                    'resprate_apache', 'temp_apache', 'd1_diasbp_max', 'd1_diasbp_min',
                                                    'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min',
                                                    'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min',
                                                    'd1_temp_max', 'd1_temp_min', 'd1_glucose_max', 'd1_glucose_min']).reset_index(drop=True)
scaled_data_df

,age,bmi,height,weight,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,map_apache,...,d1_mbp_max,d1_mbp_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,d1_glucose_max,d1_glucose_min
0,0.297782,-0.793148,0.992934,-0.419735,-0.404177,0.431091,0.077494,0.091107,0.579069,-1.109528,...,-0.720101,-1.222879,0.425570,-1.640611,-0.698524,-1.165315,3.675103,1.264498,-0.116867,-0.095852
1,0.847794,-0.230513,-0.898754,-0.567104,-2.417341,-1.789454,-1.773937,-0.161242,0.644658,-0.967542,...,0.768477,-1.769419,0.425570,-2.043081,0.390071,-1.469907,-1.486185,-1.518845,-0.379037,0.414135
2,1.092244,-0.804818,-0.423502,-0.905654,0.602404,0.431091,0.694638,0.277626,0.447890,-0.636242,...,-0.960194,1.373186,0.425570,0.472356,0.351192,-0.606897,0.951090,-1.916465,0.076911,-0.659522
3,0.236669,-0.214467,1.943437,0.619812,0.602404,0.431091,0.694638,-0.040553,0.415096,1.020258,...,1.104607,1.099916,-1.671398,0.069886,0.934368,0.560705,-0.769340,0.469257,-0.253651,0.333611
4,-0.252231,3.372639,-0.423502,2.874156,0.602404,0.431091,0.694638,0.409287,1.070988,1.209573,...,0.624421,2.261314,0.425570,-0.332584,0.079044,1.880603,-0.195863,-1.651385,0.213695,0.440976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38181,0.358894,0.069880,1.943437,0.962345,-2.417341,-1.049272,-1.773937,-0.051525,-1.749348,-0.565249,...,-1.248306,-0.061481,0.425570,0.572974,-0.854038,0.357644,-0.625970,-0.060903,-0.196658,0.092038
38182,0.236669,0.369629,0.759967,0.719385,-2.417341,-0.309091,-1.773937,-0.479421,0.611863,-0.872885,...,-0.191896,-0.812974,-0.273420,-0.433201,-1.126186,-1.013019,-1.342816,-0.856144,-0.333442,0.092038
38183,1.153357,1.679250,-3.023408,-0.579053,0.602404,-0.309091,-1.773937,-0.216100,-0.044029,0.830944,...,0.864514,0.416741,-0.972409,-0.030731,1.012125,0.053052,-0.769340,-0.723604,-0.219455,0.279928
38184,1.520032,-1.273530,-0.190536,-1.267703,-1.410759,-0.309091,-1.773937,0.244711,0.480685,1.185909,...,-0.047840,0.485059,0.425570,0.874826,-0.037591,0.154582,-0.912709,0.204177,0.042715,0.870439


In [ ]:
# identify categorical columns to be encoded
ctgy_cols = []
for i, v in clean_diabetes_df.dtypes[clean_diabetes_df.dtypes == 'object'].items():
  ctgy_cols.append(i)

dummies = pd.get_dummies(clean_diabetes_df[ctgy_cols]).astype(int).reset_index(drop=True)
dummies

,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M
0,0,0,1,0,0,0,0,1
1,0,0,1,0,0,0,1,0
2,0,0,1,0,0,0,1,0
3,0,0,1,0,0,0,0,1
4,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
38181,0,0,1,0,0,0,0,1
38182,0,0,1,0,0,0,0,1
38183,0,0,0,0,0,1,1,0
38184,0,1,0,0,0,0,0,1


In [ ]:
# save non-transformed/scaled data to a new dataframe to remove index and improve readability
binary_data = clean_diabetes_df[['apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_post_operative',
       'arf_apache', 'intubated_apache', 'ventilated_apache', 'aids', 'cirrhosis', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma', 'diabetes_mellitus']].reset_index(drop=True)

In [ ]:
# combine the transformed, scaled, and unmodified data
final_processed_df = pd.concat([dummies, scaled_data_df, binary_data], axis = 1)
final_processed_df

,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,age,bmi,...,arf_apache,intubated_apache,ventilated_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,diabetes_mellitus
0,0,0,1,0,0,0,0,1,0.297782,-0.793148,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,1,0,0.847794,-0.230513,...,0,0,1,0,0,0,0,0,0,1
2,0,0,1,0,0,0,1,0,1.092244,-0.804818,...,0,1,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,1,0.236669,-0.214467,...,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,1,0,-0.252231,3.372639,...,0,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38181,0,0,1,0,0,0,0,1,0.358894,0.069880,...,0,0,1,0,0,0,0,0,0,0
38182,0,0,1,0,0,0,0,1,0.236669,0.369629,...,0,1,1,0,0,0,0,0,0,1
38183,0,0,0,0,0,1,1,0,1.153357,1.679250,...,0,0,0,0,0,0,0,0,0,0
38184,0,1,0,0,0,0,0,1,1.520032,-1.273530,...,0,1,1,0,0,0,0,0,0,0


In [ ]:
# split preprocessed data
X = final_processed_df.drop(columns=['diabetes_mellitus'])
y = final_processed_df['diabetes_mellitus']

# split further into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Compile, Train, and Evaluate Model

In [ ]:
# Modelmodel - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
# setting initial layer with 90 neurons (~2x # of inputs)
# input_dim is 46 because of the transformed # of input columns after get_dummies
nn.add(tf.keras.layers.Dense(units=90, activation='relu', input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu'))

# Output layer
# use sigmoid activation function here because predicting binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 90)                  │           4,230 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 40)                  │           3,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              41 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,911 (30.90 KB)

 Trainable params: 7,911 (30.90 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
trained = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8135 - loss: 0.3946
Epoch 2/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8094 - loss: 0.3987
Epoch 3/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8093 - loss: 0.3979
Epoch 4/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8158 - loss: 0.3869
Epoch 5/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8152 - loss: 0.3933
Epoch 6/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8156 - loss: 0.3889
Epoch 7/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8189 - loss: 0.3872
Epoch 8/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8132 - loss: 0.3912
Epoch 9/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8165 - loss: 0.3894
Epoch 10/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8112 - loss: 0.3940
Epoch 11/100
895/895 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8079 - loss: 0.3971
Epoch 12/100
895/895 ━━━━━━━━━━━━━━━━━━━━